# Gate String Lists Tutorial
This tutorial will show you how to create and use `GateString` objects, which represent an ordered sequence (or "string") of quantum gate operations.  In almost all cases, you'll be using a list (or even a list of lists!) of `GateString`s, so we'll often be talking about "gate string lists".  You may have noticed in Tutorial 00 that we had to generate gate string lists of "fiducials" and "germs" in order to populate the data template file which GST analyzes to produce its estimates.

A `GateString` object is nearly identical, and sometimes interchangeable, with a Python tuple of gate labels (i.e. the names beginning with `G` that label the gate operations in a `GateSet`).  They can be accessed and operated upon just as a standard Python tuple.  The primary difference between a `GateString` and a tuple is that a `GateString` also contains a "string representation" of the gate sequence.  This string representation gets carried along for the ride until it's needed, typically when writing a the gate string to a file.  The string representation must *evaluate*, using pyGSTi's allowed text format for gate strings (see below), to the tuple-of-gate-labels, or "tuple representation".  The string representation is intended to contain a compact and intuitive human-readable form of the gate sequence that is used for display purposes.  For example, the gate string `('Gx','Gx','Gx','Gx','Gx')` might have the string representation `"Gx^5"`.  If needed, the tuple and string representations of any `GateString` can be accessed via `.tup` and `.str` respectively.

Gate strings are central to Gate Set Tomography, as they describe both real and "simulated" experiments.  A `GateString`'s ordered sequence tells the experimentalist which gates they must execute on their hardware and likewise what order to compose (i.e. multiply) the gate matrices contained in a `GateSet`.  The outcomes of an experiment correspond to different **SPAM labels** (c.f. the gate set tutorial), and so by repeating an experiment one obtains counts and thereby frequencies for each SPAM label.  Given a `GateSet` one can obtain corresponding probabilities by muliplying gate matrices and contracting the product between the state preparation and POVM effect vectors associated with each SPAM label.  

The **ordering direction** is important.  The elements of a `GateString` are read from **left-to-right**, meaning the first (left-most) gate label is performed first.  This is very natural for experiments since one can read the gate string as a script, executing each gate as one reads from left to right.  However, since we insist on "normal" matrix multiplication conventions, the ordering of the matrix product is *reversed* from that of the gate string.  For example, the gate string `('Ga','Gb','Gc')`, in which Ga is performed first, corresponds to the matrix product $G_c G_b G_a$.  The probability of this gate string for a SPAM label associated with the (column) vectors ($\rho_0$,$E_0$) is given by $E_0^T G_c G_b G_a \rho_0$, which can be interpreted as "prepare state 0 first, then apply gate A, then B, then C, and finally measure effect 0".  While this nuance is typically hidden from the user (the `GateSet` functions which compute products and probabilities from `GateString`s perform the order reversal internally), it becomes very important if you plan to perform such products by hand. 

We'll now go over some examples of how to create and use a single `GateString`.

In [1]:
from __future__ import print_function

In [2]:
import pygsti # the main pyGSTi module
import pygsti.construction as pc  #shorthand
from pygsti.construction import std1Q_XY #a standard gateset & peripherals

## A simple example: the single `GateString`
The cell below show how to create a `GateString` object from a tuple, optionally with a corresponding string representation.  It demonstrates how to access the tuple and string representations directly, and the tuple-like operations that can be performed on a `GateString`.

In [3]:
#Construction of a GateString
s1 = pygsti.objects.GateString( ('Gx','Gx') ) # from a tuple
s2 = pygsti.objects.GateString( ('Gx','Gx'), "Gx^2" ) # from tuple and string representations (must match!)
s3 = pygsti.objects.GateString( None, "Gx^2" ) # from just a string representation

#All of these are equivalent (even though their string representations aren't -- only tuples are compared)
assert(s1 == s2 == s3)

#Printing displays the string representation
print("Printing")
print("s1 = %s" % s1)
print("s2 = %s" % s2)
print("s3 = %s" % s3, end='\n\n')

#Casting to tuple displays the tuple representation
print("Printing tuple(.)")
print("s1 =", tuple(s1))
print("s2 =", tuple(s2))
print("s3 =", tuple(s3), end='\n\n')

#Access to tuple or string representation directly:
print("s1.tup =", s1.tup, ",  s1.str = ", s1.str, end='\n\n')

#Operations
assert(s1 == ('Gx','Gx')) #can compare with tuples
s4 = s1+s2 #addition (note this concatenates string reps)
s5 = s1*3  #integer-multplication (note this exponentiates in string rep)
print("s1 + s2 = ",s4, ", tuple = ", tuple(s4))
print("s1*3    = ",s5, ", tuple = ", tuple(s5), end='\n\n')

Printing
s1 = GxGx
s2 = Gx^2
s3 = Gx^2

Printing tuple(.)
s1 = ('Gx', 'Gx')
s2 = ('Gx', 'Gx')
s3 = ('Gx', 'Gx')

s1.tup = ('Gx', 'Gx') ,  s1.str =  GxGx

s1 + s2 =  GxGxGx^2 , tuple =  ('Gx', 'Gx', 'Gx', 'Gx')
s1*3    =  (GxGx)^3 , tuple =  ('Gx', 'Gx', 'Gx', 'Gx', 'Gx', 'Gx')



## List Construction Functions:  `pygsti.construction` and `create_gatestring_list`
Usually you'll be working with entire lists of `GateString` objects which define some part of the experiments utilized by Gate Set Tomography.  pyGSTi provides several functions for constructing gate string lists, which we not demonstrate.

The workhorse function is `pygsti.construction.create_gatestring_list`, which executes its positional arguments within a nested loop given by iterable keyword arguments.  That's a mouthful, so let's look at a few examples:

In [4]:
As = [('a1',),('a2',)]
Bs = [('b1','b2'), ('b3','b4')]

def rep2(x):
    return x+x

list1 = pc.create_gatestring_list("a", a=As)
list2 = pc.create_gatestring_list("a+b", a=As, b=Bs, order=['a','b'])
list3 = pc.create_gatestring_list("R(a)+c", a=As, c=[('c',)], R=rep2)

print("list1 = %s" % list(map(tuple, list1)))
print("list2 = %s" % list2)
print("list3 = %s" % list(map(str,list3)))

list1 = [('a1',), ('a2',)]
list2 = [GateString(a1b1b2), GateString(a1b3b4), GateString(a2b1b2), GateString(a2b3b4)]
list3 = ['a1a1c', 'a2a2c']


Many of the gate sequences used by Gate Set Tomography are composed of three parts.  A "preparation fiducial" sequence is followed by a "repeated germ" sequence, which is followed by a "measurement fiducial" sequence.  We won't get into why this structure is used, but simply use this fact to motivate looking at gate strings of the form $f_1 + R(g) + f_2$, where the $f_1$ and $f_2$ fiducial sequences are simple short sequences are $R(g)$ is a possibly long sequence that is generated by repeating a short sequence $g$ called a "germ".

It is possible to generate "repeated germ" sequences in several ways using the functions **`pygsti.construction.repeat_`*xxx* **.  In modern GST, germ sequences are always repeated an *integer* number of times rather than being truncated to a precise length, so `repeat_with_max_length` is used instead of `repeat_and_truncate`.  Below we demonstrate the use of these functions.

In [5]:
print(pc.repeat_and_truncate(('A', 'B', 'C'), 5)) #args (x,N): repeat x until it is exactly length N

print(pc.repeat_with_max_length(('A', 'B', 'C'), 5)) #args (x,N): repeat x the maximum integer number of times so len(x) < N

print(pc.repeat_count_with_max_length(('A', 'B', 'C'), 5)) #args (x,N): the maximum integer number of times so len(x) < N

('A', 'B', 'C', 'A', 'B')
('A', 'B', 'C')
1


We can combine a repeated germ sequence between two fiducial sequences using `create_gatestring_list`.  This demonstrates the power of the `create_gatestring_list` to perform nested loops.  We also introduce the "bulk-conversion" function `gatestring_list`, which creates a list of `GateString` objects from a list of tuples.

In [6]:
fids  = pc.gatestring_list( [ ('Gf0',), ('Gf1',)    ] ) #fiducial strings
germs = pc.gatestring_list( [ ('G0',), ('G1a','G1b')] ) #germ strings

gateStrings1 = pc.create_gatestring_list("f0+germ*e+f1", f0=fids, f1=fids,
                                       germ=germs, e=2, order=["germ","f0","f1"])
print("gateStrings1 = \n", "\n".join(map(str,gateStrings1)),"\n")

gateStrings2 = pc.create_gatestring_list("f0+T(germ,N)+f1", f0=fids, f1=fids,
                                        germ=germs, N=3, T=pc.repeat_and_truncate,
                                        order=["germ","f0","f1"])

print("gateStrings2 = \n", "\n".join(map(str,gateStrings2)),"\n")

gateStrings3 = pc.create_gatestring_list("f0+T(germ,N)+f1", f0=fids, f1=fids,
                                        germ=germs, N=3, T=pc.repeat_with_max_length,
                                        order=["germ","f0","f1"])
print("gateStrings3 = \n", "\n".join(map(str,gateStrings3)), "\n")

gateStrings1 = 
 Gf0(G0)^2Gf0
Gf0(G0)^2Gf1
Gf1(G0)^2Gf0
Gf1(G0)^2Gf1
Gf0(G1aG1b)^2Gf0
Gf0(G1aG1b)^2Gf1
Gf1(G1aG1b)^2Gf0
Gf1(G1aG1b)^2Gf1 

gateStrings2 = 
 Gf0G0G0G0Gf0
Gf0G0G0G0Gf1
Gf1G0G0G0Gf0
Gf1G0G0G0Gf1
Gf0G1aG1bG1aGf0
Gf0G1aG1bG1aGf1
Gf1G1aG1bG1aGf0
Gf1G1aG1bG1aGf1 

gateStrings3 = 
 Gf0(G0)^3Gf0
Gf0(G0)^3Gf1
Gf1(G0)^3Gf0
Gf1(G0)^3Gf1
Gf0(G1aG1b)Gf0
Gf0(G1aG1b)Gf1
Gf1(G1aG1b)Gf0
Gf1(G1aG1b)Gf1 



In addition to `create_gatestring_list`, the **`pygsti.construction.list_`*xxx* ** functions provide ways of constructing common gate string lists.  The example below shows how to construct all possible gate strings within a certain length range, as well as how to construct the set of gate strings needed to run Linear Gate Set Tomography given a set of fiducial strings. 

In [7]:
myGates = [ 'Gx', 'Gy' ]  #gate labels -- often just gateset.gates.keys()
allStringsInLengthRange = pc.list_all_gatestrings(myGates, minlength=0, maxlength=2)
print("\nAll strings using %s up to length 2 = \n" \
    % str(myGates), "\n".join(map(str,allStringsInLengthRange)))


All strings using ['Gx', 'Gy'] up to length 2 = 
 {}
Gx
Gy
GxGx
GxGy
GyGx
GyGy


In [8]:
myFiducialList = pc.gatestring_list([ ('Gf1',), ('Gf2',) ])  #list of fiducials

#Create spam specs which is just a tuple of two lists SpamSpec objects: one list  
# for preparation, the other for measurment. Each SpamSpec object associates a 
# fiducial gate string with a state prep (for preparation fiducials)
# or a POVM effect (for measurement fiducials).  In this example, since we're 
# just interested in the LGST strings, the state preps and POVM effects do not
# enter and are irrelevant.
mySpecs = pc.build_spam_specs(fiducialGateStrings=myFiducialList) 

lgstStrings = pc.list_lgst_gatestrings(mySpecs,myGates)

print("\nLGST strings = \n","\n".join(map(str,lgstStrings)))


LGST strings = 
 Gf1
Gf2
Gf1Gf1
Gf1Gf2
Gf2Gf1
Gf2Gf2
Gf1(Gx)Gf1
Gf1(Gx)Gf2
Gf2(Gx)Gf1
Gf2(Gx)Gf2
Gf1(Gy)Gf1
Gf1(Gy)Gf2
Gf2(Gy)Gf1
Gf2(Gy)Gf2


## Generating gate string lists for GST (and further tutorials)
As a final full-fledged example we demonstrate functions which generate gate string lists for running extended LGST (eLGST or EXLGST) and long-sequence GST (LSGST) from lists of gates, fiducials, germs, and maximum lengths.  eLGST and LSGST are two different algorithms for performing Gate Set Tomography (more detail will be given on these in the Algorithms tutorial).  The important different between the two for our purposes is that eLGST does *not* include fiducial-string prefixes or postfixes in its lists whereas LSGST does.  The following example functions are very similar to `pygsti.construction.make_lsgst_lists`, `pygsti.construction.make_elgst_lists`, and can be copied verbatim then modified in many circumstances to provide customized gate string generation.

Both functions product a *list* of lists of `GateString` objects.  As we'll see in later tutorials, eLGST and LSGST Gate Set Tomography algorithms utilize an iterative approach whereby longer and longer gate strings are used in each successive iteration.  Each list in the list-of-lists returned by these functions specifies the gate sequences to use during the corresponding iteration.  Thus, each successive list contains longer gate strings.  Each list is generated using a maximum length, and the *list* of maximum lengths, `maxLengthList` below, specifies the maximum length of each list in the lists-of-lists.  Thus, `maxLengthList` should be an *increasing* list of integers (in practice, increasing by powers of two seems good) and the length of `maxLengthList` determines the length of the returned list-of-lists, i.e. the number of gate string lists.

In [9]:
def make_lsgst_lists(gateLabels, fiducialList, germList, maxLengthList):
    singleGates = pc.gatestring_list([(g,) for g in gateLabels])
    lgstStrings = pc.list_lgst_gatestrings(pc.build_spam_specs(fiducialList), gateLabels)
    lsgst_list = pc.gatestring_list([ () ]) #running list of all strings so far
    
    if maxLengthList[0] == 0:
        lsgst_listOfLists = [ lgstStrings ]
        maxLengthList = maxLengthList[1:]
    else: lsgst_listOfLists = [ ]
        
    for maxLen in maxLengthList:
        lsgst_list += pc.create_gatestring_list("f0+R(germ,N)+f1", f0=fiducialList,
                                           f1=fiducialList, germ=germList, N=maxLen,
                                           R=pc.repeat_with_max_length,
                                           order=('germ','f0','f1'))
        lsgst_listOfLists.append( pygsti.remove_duplicates(lgstStrings + lsgst_list) )

    print("%d LSGST sets w/lengths" % len(lsgst_listOfLists), map(len,lsgst_listOfLists))
    return lsgst_listOfLists

def make_elgst_lists(gateLabels, fiducialList, germList, maxLengthList):
    singleGates = pc.gatestring_list([(g,) for g in gateLabels])
    lgstStrings = pc.list_lgst_gatestrings(pc.build_spam_specs(fiducialList), gateLabels)
    elgst_list = pc.gatestring_list([ () ])  #running list of all strings so far
    
    if maxLengthList[0] == 0:
        elgst_listOfLists = [ singleGates ]
        maxLengthList = maxLengthList[1:]
    else: elgst_listOfLists = [ ]
        
    for maxLen in maxLengthList:
        elgst_list += pc.create_gatestring_list("R(germ,N)", germ=germList, N=maxLen,
                                           R=pc.repeat_with_max_length)
        elgst_listOfLists.append( pygsti.remove_duplicates(singleGates + elgst_list) )

    print("%d eLGST sets w/lengths" % len(elgst_listOfLists),map(len,elgst_listOfLists))
    return elgst_listOfLists

We'll now use these functions to generate some lists we'll use in other tutorials.  To do this, we'll use `pygsti.io.write_gatestring_list` to write the lists to text files with one gate string (in it's string representation) per line.

In [10]:
gates = ['Gi','Gx','Gy']
fiducials = pc.gatestring_list([ (), ('Gx',), ('Gy',), ('Gx','Gx'), ('Gx','Gx','Gx'), ('Gy','Gy','Gy') ]) # fiducials for 1Q MUB
germs = pc.gatestring_list( [('Gx',), ('Gy',), ('Gi',), ('Gx', 'Gy',),
                             ('Gx', 'Gy', 'Gi',), ('Gx', 'Gi', 'Gy',),('Gx', 'Gi', 'Gi',), ('Gy', 'Gi', 'Gi',),
                             ('Gx', 'Gx', 'Gi', 'Gy',), ('Gx', 'Gy', 'Gy', 'Gi',),
                             ('Gx', 'Gx', 'Gy', 'Gx', 'Gy', 'Gy',)] )
maxLengths = [1,2,4,8,16,32,64,128,256]    
elgst_lists = make_elgst_lists(gates, fiducials, germs, maxLengths)
lsgst_lists = make_lsgst_lists(gates, fiducials, germs, maxLengths)    

print("\nFirst 20 items for dataset generation in label : string format")
for gateString in lsgst_lists[-1][0:30]:
    print(str(gateString).ljust(20), ": ", tuple(gateString))

9 eLGST sets w/lengths <map object at 0x10a23ec18>
9 LSGST sets w/lengths <map object at 0x10a265978>

First 20 items for dataset generation in label : string format
{}                   :  ()
Gx                   :  ('Gx',)
Gy                   :  ('Gy',)
GxGx                 :  ('Gx', 'Gx')
GxGxGx               :  ('Gx', 'Gx', 'Gx')
GyGyGy               :  ('Gy', 'Gy', 'Gy')
GxGy                 :  ('Gx', 'Gy')
GxGxGxGx             :  ('Gx', 'Gx', 'Gx', 'Gx')
GxGyGyGy             :  ('Gx', 'Gy', 'Gy', 'Gy')
GyGx                 :  ('Gy', 'Gx')
GyGy                 :  ('Gy', 'Gy')
GyGxGx               :  ('Gy', 'Gx', 'Gx')
GyGxGxGx             :  ('Gy', 'Gx', 'Gx', 'Gx')
GyGyGyGy             :  ('Gy', 'Gy', 'Gy', 'Gy')
GxGxGy               :  ('Gx', 'Gx', 'Gy')
GxGxGxGxGx           :  ('Gx', 'Gx', 'Gx', 'Gx', 'Gx')
GxGxGyGyGy           :  ('Gx', 'Gx', 'Gy', 'Gy', 'Gy')
GxGxGxGy             :  ('Gx', 'Gx', 'Gx', 'Gy')
GxGxGxGxGxGx         :  ('Gx', 'Gx', 'Gx', 'Gx', 'Gx', 'Gx')
GxGxGxG

In [11]:
#Write example gatestring list files for later use
pygsti.io.write_gatestring_list("tutorial_files/Example_FiducialList.txt", fiducials,"#My fiducial strings")
pygsti.io.write_gatestring_list("tutorial_files/Example_GermsList.txt", germs,"#My germ strings")

pygsti.io.write_gatestring_list("tutorial_files/Example_GatestringList.txt",lsgst_lists[-1],"#All the gate strings to be in my dataset")
pygsti.io.write_empty_dataset("tutorial_files/Example_DatasetTemplate.txt",lsgst_lists[-1])

for l,lst in zip(maxLengths,elgst_lists):
    pygsti.io.write_gatestring_list("tutorial_files/Example_eLGSTlist%d.txt" % l,lst,
                                "# eLGST gate strings for max length %d" % l)

for l,lst in zip(maxLengths,lsgst_lists):
    pygsti.io.write_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l,lst,
                                "# LSGST gate strings for max length %d" % l)
    
#Also write the max lengths we used to file
import json
json.dump(maxLengths, open("tutorial_files/Example_maxLengths.json","w"))